In [1]:
import librosa
import pandas as pd
import numpy as np
from scipy.spatial import distance

import matplotlib.pyplot as plt
%matplotlib inline
# import os
from PIL import Image
import pathlib
import csv
import operator
import IPython.display as ipd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

from random import randrange, expovariate


import tensorflow.keras

ModuleNotFoundError: No module named 'librosa'

In [ ]:
pip install librosa

In [ ]:
path = './genres/'
genres = 'blues classical country disco hiphop jazz metal pop reggae rock'.split()

In [2]:
path = './genres/'
genres = 'blues classical country disco hiphop jazz metal pop reggae rock'.split()

In [3]:
songs_data = pd.read_csv('data.csv')

# filename and label columns are not necessary in the dataset (label can be found with the index on the song)
songs_files = songs_data.values[:,0]
songs_spectra = songs_data.drop(['filename', 'chroma_stft', 'rmse', 'spectral_centroid', 'spectral_bandwidth', 'rolloff', 'zero_crossing_rate', 'label'],axis=1).values

print(songs_data.shape)
print(songs_data.head())
print(songs_spectra.shape)

(1000, 28)
          filename  chroma_stft      rmse  spectral_centroid  \
0  blues.00093.wav     0.377690  0.065906         569.930721   
1  blues.00087.wav     0.336773  0.065906        1442.190271   
2  blues.00050.wav     0.400860  0.065906        1945.848425   
3  blues.00044.wav     0.390212  0.065906        2279.124558   
4  blues.00078.wav     0.414188  0.065906        2333.685108   

   spectral_bandwidth      rolloff  zero_crossing_rate       mfcc1  \
0          995.407125   927.427725            0.021701 -350.436188   
1         1870.534155  3083.414688            0.050889 -155.504929   
2         2082.246626  4175.874749            0.085806  -82.979019   
3         2375.102120  5198.360233            0.092570 -109.509285   
4         2227.425609  4942.811778            0.123863   -2.524338   

        mfcc2      mfcc3  ...     mfcc12     mfcc13    mfcc14     mfcc15  \
0  169.545746  31.820370  ...   1.821690  -5.970891 -5.259567  -0.229211   
1  125.638863   1.596553  ...  

In [4]:
### Initiate variables ###

nb_users = 5000
nb_songs = 1000
nb_songs_per_genre = 100
nb_genres = 10
max_listened_musics_per_users = 1000

nb_reco_per_user = 20

In [5]:
#### GENERATE USER DATA ####

# users_songs : [[[id_song, note]]] => note 1..10

users_genre = []
users_genre_part = []
users_nb_songs = []
users_songs = []

for i in range (nb_users):
    user_nb_songs = int(expovariate(0.01))
    if user_nb_songs > max_listened_musics_per_users:
        user_nb_songs = max_listened_musics_per_users
    users_nb_songs.append(user_nb_songs)
    
    user_genre = randrange(10)
    users_genre.append(user_genre)
    
    user_genre_part = randrange(7)+2 # 2 .. 8
    users_genre_part.append(user_genre_part)
    
    user_songs = []
    
    for j in range(user_nb_songs):
        note = int(expovariate(0.1)) + 1
        while note > 10:
            note = int(expovariate(0.1)) + 1
        if (randrange(10) < user_genre_part): # take music from specific genre
            user_songs.append([randrange(100)+(user_genre*100), note])
        else:
            user_songs.append([randrange(1000), note])
    
    users_songs.append(user_songs)

#np_users_songs = np.asarray(users_songs,dtype=int) => ERROR sequence to np array

print(users_genre[0])
print(users_songs[0])


7
[[743, 1], [781, 1], [719, 9], [301, 3], [927, 2], [985, 3], [323, 7], [631, 1], [156, 6], [786, 5], [570, 2], [797, 8], [714, 1], [744, 3], [382, 2], [387, 9], [793, 1], [711, 5], [787, 8], [777, 6], [770, 2], [170, 3], [446, 2], [750, 10], [714, 4], [726, 3], [106, 3], [601, 6], [809, 10], [211, 9], [706, 2], [64, 10], [201, 6], [535, 7], [778, 2], [659, 2], [776, 8], [762, 2], [100, 4], [710, 2], [459, 4], [148, 3], [727, 2], [171, 2]]


In [6]:
# Make a spectrum for each user, from each of his musics listened (depending to his rate)
users_spectra = []
for user in users_songs:
    user_spectra = []
    for song in user:
        for i in range(song[1]):
            user_spectra.append(songs_spectra[song[0]])
    np_user_spectra = np.array(user_spectra, dtype=float)
    #user_spectrum = np.mean(np_user_spectra, axis=0)
    users_spectra.append(np.mean(np_user_spectra, axis=0)) #user_spectrum

print(users_spectra[0])

C:\Users\JULES LAMA\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\JULES LAMA\Anaconda3\lib\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


[-1.31251142e+02  9.34353992e+01  2.04321915e+00  2.74916495e+01
  6.00068277e-01  8.51342597e+00 -1.71477261e+00  6.10372737e+00
 -1.90587149e+00  5.97266935e+00 -3.47693342e+00  2.17925958e+00
 -2.76052628e+00  1.87757343e-01 -1.43937703e+00  7.71560672e-01
 -1.29781995e+00  8.19757383e-01  1.15451829e-01 -2.78844219e-01]


In [7]:
# print users_mean_spectra errors
i=0
for s in users_spectra:
    print(i)
    if not isinstance(s,np.ndarray):
        print("!!!!!!!!!!!!!!!!!!!!!\n!!!!!!!!!!!!!!!!!!!!!\n!!!!!!!!!!!!!!!!!!!!!")
        print('>>>> spectre :',i,' - ',s)
        print("!!!!!!!!!!!!!!!!!!!!!\n!!!!!!!!!!!!!!!!!!!!!\n!!!!!!!!!!!!!!!!!!!!!")
    else:
        j=0
        for f in s:
            if not isinstance(f, float):
                print("!!!!!!!!!!!!!!!!!!!!!\n!!!!!!!!!!!!!!!!!!!!!\n!!!!!!!!!!!!!!!!!!!!!")
                print('>>>>',j,'<<<<')
                print("!!!!!!!!!!!!!!!!!!!!!\n!!!!!!!!!!!!!!!!!!!!!\n!!!!!!!!!!!!!!!!!!!!!")
            j+=1
    i+=1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
!!!!!!!!!!!!!!!!!!!!!
!!!!!!!!!!!!!!!!!!!!!
!!!!!!!!!!!!!!!!!!!!!
>>>> spectre : 61  -  nan
!!!!!!!!!!!!!!!!!!!!!
!!!!!!!!!!!!!!!!!!!!!
!!!!!!!!!!!!!!!!!!!!!
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
!!!!!!!!!!!!!!!!!!!!!
!!!!!!!!!!!!!!!!!!!!!
!!!!!!!!!!!!!!!!!!!!!
>>>> spectre : 84  -  nan
!!!!!!!!!!!!!!!!!!!!!
!!!!!!!!!!!!!!!!!!!!!
!!!!!!!!!!!!!!!!!!!!!
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
19

2281
2282
2283
2284
2285
2286
2287
2288
2289
2290
2291
2292
2293
2294
2295
2296
2297
2298
2299
2300
2301
2302
2303
2304
2305
2306
2307
2308
2309
2310
!!!!!!!!!!!!!!!!!!!!!
!!!!!!!!!!!!!!!!!!!!!
!!!!!!!!!!!!!!!!!!!!!
>>>> spectre : 2310  -  nan
!!!!!!!!!!!!!!!!!!!!!
!!!!!!!!!!!!!!!!!!!!!
!!!!!!!!!!!!!!!!!!!!!
2311
2312
2313
2314
2315
2316
2317
2318
2319
2320
2321
2322
2323
2324
2325
2326
2327
2328
2329
2330
2331
2332
2333
2334
2335
2336
2337
2338
2339
2340
2341
!!!!!!!!!!!!!!!!!!!!!
!!!!!!!!!!!!!!!!!!!!!
!!!!!!!!!!!!!!!!!!!!!
>>>> spectre : 2341  -  nan
!!!!!!!!!!!!!!!!!!!!!
!!!!!!!!!!!!!!!!!!!!!
!!!!!!!!!!!!!!!!!!!!!
2342
2343
2344
2345
2346
2347
2348
2349
2350
2351
2352
2353
2354
2355
2356
2357
2358
2359
2360
2361
2362
2363
2364
2365
2366
2367
2368
2369
2370
2371
2372
2373
2374
2375
2376
2377
2378
2379
2380
2381
2382
2383
2384
2385
2386
2387
2388
2389
2390
2391
2392
2393
2394
2395
2396
2397
2398
!!!!!!!!!!!!!!!!!!!!!
!!!!!!!!!!!!!!!!!!!!!
!!!!!!!!!!!!!!!!!!!!!
>>>> spectre : 2398  -  

4619
4620
4621
4622
4623
4624
4625
4626
4627
4628
4629
4630
4631
4632
4633
4634
4635
4636
4637
4638
4639
4640
4641
4642
4643
4644
4645
4646
4647
4648
4649
4650
4651
4652
4653
4654
4655
4656
4657
4658
4659
4660
4661
4662
4663
4664
4665
4666
4667
4668
4669
4670
4671
4672
4673
4674
4675
4676
4677
4678
4679
4680
4681
4682
4683
4684
4685
4686
4687
4688
4689
4690
4691
4692
4693
4694
4695
4696
4697
4698
4699
4700
4701
4702
4703
4704
4705
4706
4707
4708
4709
4710
4711
4712
4713
4714
4715
4716
4717
4718
4719
4720
4721
4722
4723
4724
4725
4726
4727
4728
4729
4730
4731
4732
4733
4734
4735
4736
4737
4738
4739
4740
4741
4742
4743
4744
4745
4746
4747
4748
4749
4750
4751
!!!!!!!!!!!!!!!!!!!!!
!!!!!!!!!!!!!!!!!!!!!
!!!!!!!!!!!!!!!!!!!!!
>>>> spectre : 4751  -  nan
!!!!!!!!!!!!!!!!!!!!!
!!!!!!!!!!!!!!!!!!!!!
!!!!!!!!!!!!!!!!!!!!!
4752
4753
4754
4755
4756
4757
4758
4759
4760
4761
4762
4763
4764
4765
4766
4767
4768
4769
4770
4771
4772
4773
4774
4775
4776
4777
4778
4779
4780
4781
4782
4783
4784
4785
4786


In [8]:
def find_closest_musics_not_listened(user_spectrum, musics_listened, nb_recos=nb_reco_per_user):
    distances = {}
    i=0
    for song_spectrum in songs_spectra:
        distances[i] = distance.euclidean(user_spectrum, song_spectrum)
        i+=1
    
    distances = sorted(distances.items(), key=operator.itemgetter(1))
    
    ir=0
    recos=[]
    for j in range(nb_recos):
        while distances[ir][0] in musics_listened:
            ir+=1
        recos.append(distances[ir][0])
        ir+=1
        
    return recos

In [9]:
### Finally, let's execute the reco loop
users_reco = []
i=0
for user in users_songs:
    if not isinstance(users_spectra[i],np.ndarray):
        users_reco.append([])
    else:
        user_songs = []
        for s in user:
            user_songs.append(s[0])
        users_reco.append(find_closest_musics_not_listened(users_spectra[i], user_songs))
    i+=1
print(users_reco[:5])

[[894, 254, 237, 785, 469, 791, 889, 525, 232, 756, 373, 262, 337, 741, 937, 466, 260, 995, 416, 986], [200, 51, 942, 67, 392, 290, 327, 284, 215, 916, 913, 905, 971, 268, 917, 384, 396, 19, 265, 956], [986, 305, 916, 470, 254, 426, 791, 290, 440, 894, 227, 262, 956, 923, 67, 913, 232, 423, 327, 416], [302, 352, 986, 916, 785, 384, 440, 301, 469, 305, 939, 290, 227, 67, 426, 416, 359, 254, 400, 404], [835, 21, 865, 526, 854, 855, 879, 937, 898, 1, 808, 791, 422, 470, 310, 857, 262, 551, 894, 568]]


In [10]:
def print_user(id_user):
    print(genres[users_genre[id_user]])
    print(users_genre_part[id_user])
    #print(users_songs[id_user])
    print(users_reco[id_user])

In [11]:
def create_player(id_music):
    audio_info = songs_data.loc[id_music]
    audio_path = path+audio_info['label']+'/'+audio_info['filename']
    return ipd.Audio(audio_path)

In [12]:
### VISUALISATION ###
## Les différents tableaux utiles pour l'aprentissage :
## users_songs : [[id_song1, note_user_song1], [id_song2, note_user_song2], ...]
print(users_songs[0],'\n')
## users_spectra (spectra = spectre au pluriel ;) : [ spectre user 1, spectre user 2, ... ] => [[-51.55399323  86.15279388 -20.2741909 59.04638672 ...], ...]
print(user_spectra[0],'\n')
## songs_spectra : [ spectre song 1, spectre song 2, ... ] => [[-51.55399323  86.15279388 -20.2741909 59.04638672 ...], ...]
print(songs_spectra[0],'\n')

[[743, 1], [781, 1], [719, 9], [301, 3], [927, 2], [985, 3], [323, 7], [631, 1], [156, 6], [786, 5], [570, 2], [797, 8], [714, 1], [744, 3], [382, 2], [387, 9], [793, 1], [711, 5], [787, 8], [777, 6], [770, 2], [170, 3], [446, 2], [750, 10], [714, 4], [726, 3], [106, 3], [601, 6], [809, 10], [211, 9], [706, 2], [64, 10], [201, 6], [535, 7], [778, 2], [659, 2], [776, 8], [762, 2], [100, 4], [710, 2], [459, 4], [148, 3], [727, 2], [171, 2]] 

[-4.00433922e+01  5.62070236e+01  2.36802235e+01  3.19244385e+01
  2.44971204e+00  2.57410622e+00 -2.47713041e+00  3.09297466e+00
  1.88285279e+00  3.35320282e+00  3.27849016e-03 -1.51962006e+00
 -2.45549726e+00 -5.07163620e+00 -3.67230320e+00 -7.84654379e+00
 -3.03793073e+00 -2.78411365e+00 -7.49525261e+00 -3.63001323e+00] 

[-3.50436188e+02  1.69545746e+02  3.18203697e+01  1.66828346e+01
  2.87101460e+01  1.92569733e+01  7.86339474e+00  1.00367641e+01
  4.73510647e+00  2.90746570e+00  3.91555262e+00  1.82168961e+00
 -5.97089148e+00 -5.25956726e+00

In [ ]:
### CREATION DU MODELE ###

# 
#
#  
#
#

In [64]:
### TEST RECO ###
# Choose a user id by changing this value :
user_test = 7
# and run the cell
print_user(user_test)
cpt_reco = -1

jazz
6
[242, 851, 867, 855, 838, 863, 854, 848, 918, 253, 263, 871, 298, 880, 850, 874, 258, 836, 332, 110]


In [65]:
# Checks music recommandation for the user_test (check the next music by incrementing the cell - no loop possible with the audio render)
cpt_reco+=1
create_player(users_reco[user_test][cpt_reco])

In [ ]:
#file = open('users.csv', 'a', newline='')
#with file:
#    writer = csv.writer(file)
#    writer.writerow(to_append.split())
#
#file = open('reco.csv', 'a', newline='')
#with file:
#    writer = csv.writer(file)
#    writer.writerow(to_append.split())